BVP

laplace equation
concentric balls around center (0, 0) - drichlet bc on both the surfaces
smallest eigenvalues and functions

In [1]:
import numpy as np
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc
import pyvista as pv
from dolfinx import mesh, fem, plot

#1 generate concentric balls domain of radius 5 and 1  neumann + drichlet 

In [2]:
inner_radius = 1.0
outer_radius = 5.0

In [3]:
import gmsh
from dolfinx.io import gmshio
# import meshio


def get_domain(inner_radius: float, outer_radius: float, filename: str = None):
    """
    inner_radius: float, 
    outer_radius: float
    """

    # if filename is not None:
        
        


    gmsh.initialize()
    model = gmsh.model()
    model.add("domain")
    model.setCurrent("domain")


    outer_sphere = model.occ.addSphere(0, 0, 0, radius=outer_radius, tag=1)
    inner_sphere = model.occ.addSphere(0, 0, 0, radius=inner_radius, tag=2)
    shell_dims_tags, _ = model.occ.cut( #perform boolean cut
        [(3, outer_sphere)], #target sphere
        [(3, inner_sphere)]  # tool sphere
    )
    model.occ.synchronize()

    #define boundaries

    #tag inner and outer surface
    boundary = model.getBoundary(shell_dims_tags)
    # boundary = model.getBoundary(shell_dims_tags, oriented=False)
    print(boundary)
    outer_surface = boundary[0][1]
    inner_surface = boundary[1][1]

    #add physical group
    model.addPhysicalGroup(2, [outer_surface], tag = 1)
    model.addPhysicalGroup(2, [inner_surface], tag = 2)
    model.addPhysicalGroup(3, [shell_dims_tags[0][1]], tag = 3) #volume
    

    #set mesh options and generate mesh
    gmsh.option.setNumber("Mesh.MeshSizeFactor", 0.2)
    # gmsh.option.setNumber("Mesh.MeshSizeMin", 0.1)
    # gmsh.option.setNumber("Mesh.MeshSizeMax", 0.1)
    # model.occ.synchronize()
    model.geo.synchronize()
    model.mesh.generate(3)
    # gmsh.fltk.run()
    # gmsh.write("spherical_shell.msh")  # Export to file
    


    #get domain from mesh
    domain, cell_markers, facet_markers = gmshio.model_to_mesh(model=model, comm=MPI.COMM_WORLD, rank = 0, gdim = 3)

    
    
    # gmsh.write("Sphere.xmdf")

    gmsh.finalize()
    return domain, cell_markers, facet_markers

In [4]:
from dolfinx import plot
import pyvista

def plot_domain(domain):

    pyvista.start_xvfb()
    gdim = domain.topology.dim #dimensions of domain!

    domain.topology.create_connectivity(gdim, gdim)
    topology, cell_types, geometry = plot.vtk_mesh(domain, gdim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=True)
    plotter.view_xy()
    if not pyvista.OFF_SCREEN:
        plotter.show()
    else:
        figure = plotter.screenshot("fundamentals_mesh.png")



In [5]:
from dolfinx import fem
import ufl

def calculate_volume(domain, cell_markers):

    dx = ufl.Measure("dx", domain=domain, subdomain_data=cell_markers)

    # Total volume
    total_volume = fem.assemble_scalar(fem.form(1.0 * dx))
    print(f"Total volume: {total_volume:.4f}")

    # Subdomain volume (e.g., physical group 1)
    subdomain_volume = fem.assemble_scalar(fem.form(1.0 * dx(1)))
    print(f"Subdomain volume: {subdomain_volume:.4f}")


In [6]:
domain, cell_markers, facet_markers = get_domain(inner_radius, outer_radius)
# plot_domain(domain)

[(2, -2), (2, 3)]                                                                                                         
Info    : Meshing 1D...
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 70%] Meshing curve 8 (Circle)
Info    : Done meshing 1D (Wall 0.000199355s, CPU 0.00022s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Sphere, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 3 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.198732s, CPU 0.195339s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 3411 nodes...
Info    : Done tetrahedrizing 3419 nodes (Wall 0.0266753s, CPU 0.023788s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0788296s, CPU 0.071087s)
Info    : Found volume 1
Info    : Found void region
Info    : It. 0 - 0 nodes created - worst tet r

In [7]:
calculate_volume(domain, cell_markers)

Total volume: 518.6631
Subdomain volume: 0.0000


#2 Setup function space and apply boundary conditions

In [8]:
def setup_fe_space(domain):
    """Create function space and variational forms. Defining bilinear and linear forms."""
    V = fem.functionspace(domain, ("Lagrange", 1))

    u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
    a = fem.form(ufl.inner(ufl.grad(u), ufl.grad(v))*ufl.dx)
    m = fem.form(ufl.inner(u, v)*ufl.dx)

    return V, a, m


In [9]:
V, a, m = setup_fe_space(domain)

preparing boundary conditions
Drichlet BC u = 0

In [10]:
from petsc4py import PETSc
def on_boundary(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2 + x[2]**2), outer_radius)

def get_bcs(V):
    boundary_dofs = fem.locate_dofs_geometrical(V, on_boundary)
    bc = fem.dirichletbc(PETSc.ScalarType(0), boundary_dofs, V)

    return bc


#3 assemble matrices and apply boundary conditions

In [11]:
import dolfinx.fem.petsc

def assemble_and_apply_bc(V, a, m):

    #get bcs

    bc = get_bcs(V)

    #  Assemble matrices A and M

    A = fem.petsc.assemble_matrix(a, bcs=[bc]) #bc only on stiffness matrix
    A.assemble()
    M = fem.petsc.assemble_matrix(m) # mass matrix
    M.assemble()

    # null_vec = A.createVecLeft()   #for neumann problem
    # null_vec.set(1.0)
    # null_vec.normalize()
    # nullspace = PETSc.NullSpace().create(vectors=[null_vec])
    # A.setNullSpace(nullspace)

    return A, M



In [12]:
A, M = assemble_and_apply_bc(V, a, m)

In [13]:
print("A nnz:", A.getInfo())
print("M nnz:", M.getInfo())


A nnz: {'block_size': 1.0, 'nz_allocated': 165217.0, 'nz_used': 165217.0, 'nz_unneeded': 0.0, 'memory': 0.0, 'assemblies': 1.0, 'mallocs': 0.0, 'fill_ratio_given': 0.0, 'fill_ratio_needed': 0.0, 'factor_mallocs': 0.0}
M nnz: {'block_size': 1.0, 'nz_allocated': 165217.0, 'nz_used': 165217.0, 'nz_unneeded': 0.0, 'memory': 0.0, 'assemblies': 1.0, 'mallocs': 0.0, 'fill_ratio_given': 0.0, 'fill_ratio_needed': 0.0, 'factor_mallocs': 0.0}


In [14]:
# print(A.norm())
# print(M.norm())

setup eigenvalue solver

In [15]:

def get_eigenvalues(A, M, n_eigen):

    eigensolver = SLEPc.EPS().create()
    eigensolver.setOperators(A, M)
    eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    # eigensolver.setDimensions(nev=n_eigen, ncv=2*n_eigen)  # Larger subspace
    eigensolver.setDimensions(nev=n_eigen)  # Larger subspace
    # eigensolver.setTolerances(tol=1e-8, max_it=1000)
    eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)

    eigensolver.setFromOptions()
    eigensolver.solve()
    n_conv = eigensolver.getConverged()

    if MPI.COMM_WORLD.rank == 0:
        print(f"Number of converged eigenvalues: {n_conv}")


    return eigensolver, n_conv

In [16]:
n_eigen = 7
eigensolver, n_conv = get_eigenvalues(A, M, n_eigen=n_eigen)

Number of converged eigenvalues: 10


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import dolfinx.plot

def plot_eigenfunctions(domain, V, eigensolver, A, nev_to_plot):
    """
    Plot the first few eigenfunctions for any given domain using matplotlib.
    Parameters:
        domain: dolfinx.mesh.Mesh
        V: dolfinx.fem.FunctionSpace
        eigensolver: SLEPc.EPS
        A: PETSc.Mat (needed for getVecs)
        n_conv: int (number of converged eigenpairs)
        nev_to_plot: int (number of eigenfunctions to plot)
    """
    if nev_to_plot == 0:
        return
    
    if MPI.COMM_WORLD.rank != 0:
        return

    topology, cell_types, geometry = plot.vtk_mesh(V)
    points = domain.geometry.x
    for i in range(nev_to_plot):
        eigval = eigensolver.getEigenvalue(i)
        error = eigensolver.computeError(i)

        print(f"Eigenvalue {i}: {eigval:.4f} (Error: {error:.2e})")

        r, _ = A.getVecs()
        eigensolver.getEigenvector(i, r)

        uh = fem.Function(V)
        uh.x.petsc_vec.setArray(r.array)
        uh.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

        # Create PyVista grid
        values = uh.x.array.real
        grid = pv.UnstructuredGrid(topology, cell_types, points)
        grid.point_data["u"] = values

        # Plot
        plotter = pv.Plotter(notebook=True)
        plotter.add_mesh(grid, show_edges=True, scalars="u", cmap="rainbow")
        plotter.view_xy()
        plotter.add_text(f"Eigenfunction {i}, λ = {eigval:.4f}", font_size=12)
        plotter.show()

# Usage example (after solving eigenproblem):
# plot_eigenfunctions(domain, V, eigensolver, A, n_conv, nev_to_plot=6)


In [ ]:
plot_eigenfunctions(domain, V, eigensolver, A, n_conv)

Eigenvalue 0: 0.4140 (Error: 6.38e-08)


Widget(value='<iframe src="http://localhost:39617/index.html?ui=P_0x79b5ef7e9550_0&reconnect=auto" class="pyvi…

Eigenvalue 1: 0.7863 (Error: 1.98e-08)


Widget(value='<iframe src="http://localhost:39617/index.html?ui=P_0x79b5e0cbdbd0_1&reconnect=auto" class="pyvi…

Eigenvalue 2: 0.7865 (Error: 2.89e-08)


...

Eigenvalue 8: 1.3419 (Error: 1.14e-09)


Widget(value='<iframe src="http://localhost:39617/index.html?ui=P_0x79b5db748690_8&reconnect=auto" class="pyvi…

Eigenvalue 9: 1.8272 (Error: 8.52e-13)


Widget(value='<iframe src="http://localhost:39617/index.html?ui=P_0x79b5db7487d0_9&reconnect=auto" class="pyvi…

: 